<a href="https://colab.research.google.com/github/sahin-arslan/ML_Final_Odevi/blob/main/MLFinalOdev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.	Size atanan veri setine tüm algoritmalar için optimizasyon uygulayınız.**


*   Normalizasyon işlemi için Min-Max yöntemini kullunıyoruz
*   Bağımlı ve bağımsız değişkenlerimizi ayırıyoruz.
*   Eğitim ve test setlerimizi ayırıyoruz(%20 test)
*   Model hiperparametlererini belirliyoruz. Burada RandomForestRegressor'den yararlanıyoruz
*   Grid Search kullanarak hiperparemater optimizasyonu yapıyoruz.
*   En iyi parametrelerimizi belirledikten sonra ekrana yazdırıyoruz.







In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Veri setini yükleyin
df = pd.read_csv('veri-seti.txt', sep="\t")

# Eksik verileri kontrol edin
print(df.isnull().sum())

# Normalizasyon
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Özellik ve hedef değişkenleri ayırın
X = df_scaled.drop('CV', axis=1)
y = df_scaled['CV']

# Veriyi eğitim ve test setlerine ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#En İyi Parametreler:

# Model ve Hiperparametreler
model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search ile Hiperparametre Optimizasyonu
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# En iyi parametreler
print(f'\nEn iyi parametre Listesi: {grid_search.best_params_}')

# En iyi model ile tahmin yapma
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Model performansını değerlendirme
mse = mean_squared_error(y_test, y_pred)
print(f'\nMSE (Tahminlerin gerçek değerlerden ne kadar saptığı:): {mse}')


NOTP    0
PGC     0
DBP     0
TST     0
INS     0
BMI     0
DPF     0
Age     0
CV      0
dtype: int64
Fitting 3 folds for each of 108 candidates, totalling 324 fits

En iyi parametre Listesi: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}

MSE (Tahminlerin gerçek değerlerden ne kadar saptığı:): 0.16973497650428213


**Sonuçların yorumlanması**
Paremetre listesini kullanarak en iyi performans için parametrelerimizi aşağıdaki gibi belirledik. Herbir kalemde ilgili parametreye ait açıklamayı aşağıda bulabilirsiniz.

1.   max_depth : None (olarak çıktı bu da ağaçımızın büyüme konusunda bir sınır koymayacağımız anlamına geliyor.)
2.   min_samples_leaf : 2 (Bir düğüm de bulunması gereken minimum dal sayısı)
3.   min_samples_split: 10 (bir sonraki düğüme dallanmak için gerekli minumun öğe sayısı)
4.   n_estimators: 50(oluşturulacak ağaç sayısı)






**2. Veri setinizi rastgele olarak %70 eğitim %30 test olacak şekilde ayırınız. Eğitim veri seti için Naive bayes sınıflandırıcısını uygulayınız. Elde ettiğiniz sonucları raporlayınız.**

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Hedef değişkeni ve bağımsız değişkenleri ayırın
X = df.drop('CV', axis=1)
y = df['CV']

# Veri setini rastgele %70 eğitim %30 test olacak şekilde ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Naive Bayes sınıflandırıcısını oluşturun ve eğitin
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapın
y_pred = nb_classifier.predict(X_test)

# Sonuçları raporlayın
accuracy = accuracy_score(y_test, y_pred)
print("Doğruluk:", accuracy)
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))

Doğruluk: 0.7445887445887446

Sınıflandırma Raporu:
               precision    recall  f1-score   support

           0       0.82      0.79      0.80       151
           1       0.62      0.66      0.64        80

    accuracy                           0.74       231
   macro avg       0.72      0.73      0.72       231
weighted avg       0.75      0.74      0.75       231



**Genel Doğruluk**
  **Doğruluk:**
    **0.7445887445887446**
      Modelinizin doğruluğu yaklaşık %74. Bu, modelinizin test verileri üzerindeki tahminlerinin %74'ünün doğru olduğunu gösterir.

**Sınıflandırma Raporu**

  **Sınıf 0 (Negatif Sınıf)**

    **Precision (Kesinlik): 0.82**
    Modelinizin sınıf 0 için yaptığı pozitif tahminlerin %82'si doğru.
    Diğer bir deyişle, model 0 sınıfında %18 oranında hata yapıyor.

  **Recall (Duyarlılık): 0.79**

  Gerçekten sınıf 0 olan örneklerin %79'unu doğru bir şekilde tahmin ediyor.
  Model, sınıf 0 olan örneklerin %21'ini kaçırıyor.

  **F1-Score: 0.80**

  Precision ve recall'un harmonik ortalaması. Bu değer, modelin sınıf 0 için genel performansını özetler.

**Sınıf 1 (Pozitif Sınıf)**
  **Precision (Kesinlik): 0.62**

  Modelinizin sınıf 1 için yaptığı pozitif tahminlerin %62'si doğru.
  Bu, sınıf 1 tahminlerinde %38 oranında hata yapıldığını gösterir.

  **Recall (Duyarlılık): 0.66**

  Gerçekten sınıf 1 olan örneklerin %66'sını doğru tahmin ediyor.
  Model, sınıf 1 olan örneklerin %34'ünü kaçırıyor.

  **F1-Score: 0.64**
  Precision ve recall'un harmonik ortalaması. Bu değer, modelin sınıf 1 için genel performansını özetler.

**Genel Performans**
  **Accuracy (Doğruluk):** 0.74
  Genel doğruluk, modelin tüm test verisi üzerindeki doğruluk oranını gösterir.

  **Macro Average (Makro Ortalama):**
  Precision: 0.72
  Recall: 0.73
  F1-Score: 0.72
  Makro ortalama, her iki sınıfın performansını eşit ağırlıkla alır ve ortalama performansı gösterir.

  **Weighted Average (Ağırlıklı Ortalama):**
  Precision: 0.75
  Recall: 0.74
  F1-Score: 0.75
  Ağırlıklı ortalama, her sınıfın veri setindeki temsil oranına göre ağırlıklandırılarak hesaplanır.


**Yorum ve Öneriler**
  Dengesizlik: Precision ve recall değerleri arasında belirgin bir fark var. Sınıf 1 için precision ve recall değerleri, sınıf 0'a göre daha düşük. Bu, modelin sınıf 1'i sınıf 0'dan daha az iyi tanıdığı anlamına gelir.

  **Model Geliştirme:**

  Özellik Seçimi: Daha fazla özellik eklemeyi veya mevcut özellikleri iyileştirmeyi düşünebilirsiniz.
  Veri Dengesizliği: Eğer veri setinizde sınıf dengesizliği varsa, bu durumu dengelemek için çeşitli teknikler (SMOTE, class weighting) kullanabilirsiniz.
  Farklı Algoritmalar: Farklı makine öğrenimi algoritmalarını (Random Forest, Support Vector Machine, XGBoost) deneyerek performansı karşılaştırabilirsiniz.
  Hiperparametre Optimizasyonu: Mevcut modelinizin hiperparametrelerini optimize ederek performansı artırabilirsiniz.
  Sonuç olarak, modeliniz sınıf 0'ı iyi bir şekilde tanırken, sınıf 1'de performans düşüklüğü yaşıyor. Bu durum, modelin sınıf dengesizliği veya sınıf 1'e ait örneklerin daha az belirgin özelliklere sahip olmasından kaynaklanabilir. Bu nedenle, veri dengesizliği ile başa çıkmak ve modelin performansını artırmak için yukarıdaki önerilere başvurabilirsiniz.

 **3.	Veri setinizi rastgele olarak %70 eğitim %30 test olacak şekilde ayırınız. Eğitim veri seti için K-en yakın komşuluk sınıflandırıcısını uygulayınız. En iyi k değerini belirleyerek Elde ettiğiniz sonuçları raporlayınız.**  



In [3]:
from sklearn.neighbors import KNeighborsClassifier


# Separate features (X) and target variable (y)
X = df.drop('CV', axis=1)
y = df['CV']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define a range of k values to test
k_values = range(1, 21)

# Initialize lists to store accuracy scores for each k value
accuracy_scores = []

# Iterate through the k values and train the KNN classifier
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Find the k value with the highest accuracy
best_k = k_values[accuracy_scores.index(max(accuracy_scores))]

# Print the results
print(f"Best k value: {best_k}")
print(f"Highest accuracy: {max(accuracy_scores)}")

Best k value: 13
Highest accuracy: 0.7445887445887446


**Sonuçların Yorumlanması**

**En İyi **
**𝑘**
**k Değeri: 13**  

𝑘
k değeri 13 olduğunda, modelinizin performansı en yüksek seviyede. Bu, KNN algoritmasının, eğitim verilerindeki en yakın 13 komşuyu kullanarak en iyi sonucu verdiği anlamına gelir.

**En Yüksek Doğruluk: 0.7445887445887446**

Modelinizin doğruluğu yaklaşık %74. Bu, test veri setinde modelin yaptığı tahminlerin %74'ünün doğru olduğu anlamına gelir.

**Değerlendirme ve Öneriler**

**Model Performansı:**

%74 doğruluk skoru, birçok makine öğrenimi problemi için makul bir performans olarak değerlendirilebilir. Ancak, daha yüksek doğruluk oranları elde etmek için çeşitli iyileştirmeler yapılabilir.

**KNN Algoritmasının Özellikleri:**

KNN algoritması, yüksek boyutlu veri setlerinde performans kaybı yaşayabilir. Bu durumda, özellik seçimi veya boyut indirgeme yöntemleri kullanarak performans artırılabilir.
KNN'nin performansı ayrıca veri setindeki örneklerin yoğunluğuna ve veri setindeki gürültüye karşı da hassastır. Bu yüzden veri ön işleme adımları önemlidir.

**Diğer Modellerle Karşılaştırma:**

KNN algoritması dışında, başka makine öğrenimi algoritmaları (örneğin, Decision Trees, Random Forests, SVM, Logistic Regression) kullanarak model performansınızı karşılaştırabilirsiniz. Bu, en iyi modeli belirlemenize yardımcı olabilir.

**Hiperparametre Optimizasyonu:**

KNN'nin
𝑘
k parametresi dışında, mesafe metriği gibi başka hiperparametrelerini de optimize edebilirsiniz. Bu, modelin performansını artırabilir.

**K-Fold Çapraz Doğrulama:**

Modelinizin doğruluğunu daha iyi değerlendirmek için K-Fold çapraz doğrulama yöntemini kullanabilirsiniz. Bu yöntem, veri setini birden fazla kez bölerek, modelin performansını daha genel bir şekilde değerlendirmenize olanak tanır.


Özetle, elde ettiğiniz sonuçlar doğrultusunda KNN modeli ile %74 doğruluk skoru yakalamışsınız. En iyi performansı veren
𝑘
k değeri 13 olarak belirlenmiş. Daha yüksek performans elde etmek için diğer algoritmaları denemek ve modelin hiperparametrelerini optimize etmek faydalı olabilir.

**4.	Veri setinizi rastgele olarak %70 eğitim %30 test olacak şekilde ayırınız. Eğitim veri seti için Multi-Layer Perceptron (MLP) ve Support Vector Machines (SVM) sınıflandırıcılarını uygulayınız. Eğitim ve test adımlarında elde ettiğiniz sonucları raporlayınız.**

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Separate features (X) and target variable (y)
X = df.drop('CV', axis=1)
y = df['CV']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# MLP Classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Training Accuracy: {mlp.score(X_train, y_train)}")
print(f"MLP Testing Accuracy: {accuracy_mlp}")

# SVM Classifier
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Training Accuracy: {svm.score(X_train, y_train)}")
print(f"SVM Testing Accuracy: {accuracy_svm}")

MLP Training Accuracy: 0.7914338919925512
MLP Testing Accuracy: 0.7229437229437229
SVM Training Accuracy: 0.7802607076350093
SVM Testing Accuracy: 0.7359307359307359


Elde ettiğiniz sonuçları yorumlayarak, MLPClassifier ve SVM modellerinin performanslarını karşılaştıralım:

**MLPClassifier (Multi-Layer Perceptron)**
•	Training Accuracy (Eğitim Doğruluğu): 0.7914338919925512
o	Model, eğitim verisi üzerinde yaklaşık %79 doğruluğa sahip. Bu, modelin eğitim verisindeki örneklerin %79'unu doğru bir şekilde sınıflandırdığı anlamına gelir.
•	Testing Accuracy (Test Doğruluğu): 0.7229437229437229
o	Model, test verisi üzerinde yaklaşık %72 doğruluğa sahip. Bu, modelin daha önce görmediği test verisindeki örneklerin %72'sini doğru bir şekilde sınıflandırdığı anlamına gelir.

**SVM (Support Vector Machine)**
•	Training Accuracy (Eğitim Doğruluğu): 0.7802607076350093
o	Model, eğitim verisi üzerinde yaklaşık %78 doğruluğa sahip.
•	Testing Accuracy (Test Doğruluğu): 0.7359307359307359
o	Model, test verisi üzerinde yaklaşık %73 doğruluğa sahip.

**Karşılaştırma ve Değerlendirme**
Eğitim Doğruluğu
•	MLP: %79
•	SVM: %78
Eğitim verisi üzerinde, MLPClassifier biraz daha yüksek bir doğruluğa sahip. Bu, MLP modelinin eğitim verisindeki örnekleri daha iyi öğrendiği anlamına gelebilir.

**Test Doğruluğu**
•	MLP: %72
•	SVM: %73
Test verisi üzerinde, SVM modeli MLP modeline göre biraz daha yüksek bir doğruluğa sahip. Bu, SVM modelinin daha önce görmediği verileri biraz daha iyi genelleme yeteneğine sahip
